In [17]:
# imports
import os
import json
import re

import torch 
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import numpy as np

from MDP import MDP

import stable_baselines3
import sb3_contrib

import gym

In [18]:
# check torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080'

In [19]:
# create Neural Network

class Net(nn.Module):
    """
    input : 2 X 4 X 4 grid
    label : Move [0;6]
    """
    def __init__(self):
        super(Net, self).__init__()
        # first layer: input
        self.conv1 = nn.Conv2d(2, 8, 2)

        #second layer : 2nd convolution
        self.conv2 = nn.Conv2d(8, 16, 2)

        self.conv3 = nn.Conv2d(16, 32, 2)

        self.fc1 = nn.Linear(32, 32)

        self.out = nn.Linear(32, 6)


    def forward(self, x):
        x = x.float()

        x = F.relu(self.conv1(x))

        x = F.relu(self.conv2(x))

        x = F.relu(self.conv3(x))

        x = torch.flatten(x,start_dim=1)

        x = F.relu(self.fc1(x))

        x = self.out(x)
    
        return x    

In [20]:
#creating model
net = Net()
net.cuda()
print(net)

params = list(net.parameters())
print(f"number of parameters: {len(params)}")

#loss function
loss = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(net.parameters())
optimizer

Net(
  (conv1): Conv2d(2, 8, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (fc1): Linear(in_features=32, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=6, bias=True)
)
number of parameters: 10


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [21]:
net.load_state_dict(torch.load("Net"))

<All keys matched successfully>

In [22]:
#custom environment
from gym import spaces

class Gridworld(gym.Env):

    metadata = {"render.modes" : ["human"]}

    def __init__(self, dir = ["data", "data_easy", "data_medium"], type = ["train"], lambda1 = 0.01, lambda2 = 0.1, lambda3 = 1) -> None:
        super(Gridworld, self).__init__()
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(low = 0, high = 10, shape = (2, 4, 4))

        #available MDPs
        self.dir = dir
        self.type = type
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3

        self.actions = ["move", "turnLeft", "turnRight", "pickMarker", "putMarker", "finish"]

    def reset(self):
        nextDir, nextType = np.random.choice(self.dir), np.random.choice(self.type)
        nexti = np.random.choice(os.listdir(os.sep.join(["datasets", nextDir, nextType, "task"])))
        nexti = re.sub(r"\D", "", nexti)
        self.currentMDP = MDP(nextDir, nextType, nexti, lambda1= self.lambda1, lambda2 = self.lambda2, lambda3 =self.lambda3)
        self.steps = 0
        return self.currentMDP.get_current_state()

    def step(self, action):
        nextState, rew, done, info = self.currentMDP.sample_next_state_and_reward(self.actions[action])
        self.steps += 1
        if self.steps > 500:
            return nextState, -1, True, info 

        return nextState, rew -0.025, done, info 
        
    def render(self):
        self.currentMDP.print_grid()

    def close(self):
        pass
    
    def action_masks(self):
        mat = self.currentMDP.get_current_state()
        if np.array_equal(mat[0], mat[1]):
            return np.array([0,0,0,0,0,1])
        
        return self.currentMDP.action_mask()

    # functions bellow are only used for inheritance 
    def get_MDP(self):
        return self.currentMDP

    def get_MDP_name(self):
        return self.nextDir, self.nextType, self.nexti

In [23]:
np.random.seed(123)

def test_RL_models(model):
    for dir, num in zip(["data_easy", "data_medium", "data"], [80, 24, 480]):
        print("current data: " + dir)
        totalRew = 0
        valDataset = Gridworld(dir = [dir], type = ["val"], lambda1=0, lambda2=0)
        correct, total = 0,num*10
        for task in range(int(total)):
            if task % num == num-1:
                print(f"{(task+1) / total *100} %, running acc: {(correct*100)/(task+1)}, average reward: {totalRew / task}")
            currMDP = valDataset.reset()
            done = False
            steps = 0
            while not done and steps < 50:
                action = model.predict(currMDP, action_masks = valDataset.action_masks(), deterministic = True)[0]
                currMDP, rew, done, _ = valDataset.step(action)
                totalRew += rew
                if rew > 0:
                    correct += 1
                steps += 1
        

In [24]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.env_util import make_vec_env

import torch.nn.functional as F
from torch import nn

class CustomFeatureExtractorTorch(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 32):
        super().__init__(observation_space, features_dim)

        self.conv1 = net.conv1
        self.conv2 = net.conv2
        self.conv3 = net.conv3
        

        #additional convolutions


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, start_dim=1)
        return x

net_arch = [
    32, 16, 8,
    dict(vf = [16, 8, 4], pi = [16, 8, 4])
]

policy_kwargs = dict(
    features_extractor_class = CustomFeatureExtractorTorch,
    net_arch = net_arch
)

In [25]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy

In [26]:
net_arch = [
    128, 64, 16, 8,
    dict(vf = [8, 4, 2], pi = [8, 4, 2])
]

policy_kwargs = dict(
    features_extractor_class = CustomFeatureExtractorTorch,
    net_arch = net_arch
)

In [30]:
FinalEnv = make_vec_env(Gridworld, n_envs= 4, env_kwargs={"lambda1" : 0, "lambda2" : 0, "lambda3": 5, "dir" : ["data_easy", "generated_easy", "data_medium", "generated_med", "data"]})

FinalModel = MaskablePPO(MaskableActorCriticPolicy, FinalEnv, policy_kwargs= policy_kwargs,  verbose = 1, n_steps= 500)

FinalModel.learn(2 * 1e5)

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2        |
|    ep_rew_mean     | 4.95     |
| time/              |          |
|    fps             | 797      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2000     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 401          |
|    ep_rew_mean          | -9.81        |
| time/                   |              |
|    fps                  | 717          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4000         |
| train/                  |              |
|    approx_kl            | 0.0009906918 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.27        |
|    explained_variance   | 0.157       

In [ ]:
FinalModel.learn(1e7)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 46.9     |
|    ep_rew_mean     | 3.76     |
| time/              |          |
|    fps             | 688      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 144         |
|    ep_rew_mean          | -0.269      |
| time/                   |             |
|    fps                  | 471         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4000        |
| train/                  |             |
|    approx_kl            | 0.008506195 |
|    clip_fraction        | 0.0579      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.659      |
|    explained_variance   | 0.542       |
|    learning_rate        | 0.

KeyboardInterrupt: 

In [170]:
test_RL_models(FinalModel)

current data: data_easy
10.0 %, running acc: 73.75, average reward: 0.35949367088607936
20.0 %, running acc: 76.25, average reward: 0.4102201257861744
30.0 %, running acc: 76.25, average reward: 0.40585774058575347
40.0 %, running acc: 72.8125, average reward: 0.3298589341692338
50.0 %, running acc: 72.25, average reward: 0.31760651629067105
60.0 %, running acc: 72.29166666666667, average reward: 0.31868475991643097
70.0 %, running acc: 73.03571428571429, average reward: 0.3357334525938539
80.0 %, running acc: 73.75, average reward: 0.35015649452262626
90.0 %, running acc: 74.86111111111111, average reward: 0.3738178025034226
100.0 %, running acc: 75.125, average reward: 0.3799436795994814
current data: data_medium
10.0 %, running acc: 45.833333333333336, average reward: -0.22826086956521824
20.0 %, running acc: 43.75, average reward: -0.297872340425538
30.0 %, running acc: 44.44444444444444, average reward: -0.28732394366196845
40.0 %, running acc: 45.833333333333336, average reward: 

In [29]:
FinalModel.load("RLModel")

In [32]:
testEnv = Gridworld()

In [166]:
curr = testEnv.reset()
testEnv.render()

[[['.' '.' '.' '<']
  ['.' '.' '#' '.']
  ['.' '#' '.' '.']
  ['.' '.' '.' '.']]

 [['.' '<' 'O' '.']
  ['.' '.' '#' '.']
  ['.' '#' '.' '.']
  ['.' '.' '.' '.']]]


In [169]:
a = FinalModel.predict(curr, action_masks= testEnv.action_masks(), deterministic= True)[0]
curr, _, done, _ = testEnv.step(a)
testEnv.render()
if done:
    print("finished")

[[['<' '.' '.' '.']
  ['.' '.' '#' '.']
  ['.' '#' '.' '.']
  ['.' '.' '.' '.']]

 [['.' '<' 'O' '.']
  ['.' '.' '#' '.']
  ['.' '#' '.' '.']
  ['.' '.' '.' '.']]]
